In [1]:
import numpy
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [2]:
from laplace_helper import p_analytical, plot_3D, L2_rel_error

In [3]:
nx = 128
ny = 128

L = 5
H = 5

x = numpy.linspace(0,L,nx)
y = numpy.linspace(0,H,ny)

dx = L/(nx-1)
dy = H/(ny-1)

p0 = numpy.zeros((ny, nx))

p0[-1,:] = numpy.sin(1.5*numpy.pi*x/x[-1])

In [4]:
def laplace2d(p, l2_target):
    '''Solves the Laplace equation using the Jacobi method 
    with a 5-point stencil
    
    Parameters:
    ----------
    p: 2D array of float
        Initial potential distribution
    l2_target: float
        Stopping criterion
        
    Returns:
    -------
    p: 2D array of float
        Potential distribution after relaxation
    '''
    
    l2norm = 1
    pn = numpy.empty_like(p)
    iterations = 0

    while l2norm > l2_target:
        pn = p.copy()
        p[1:-1,1:-1] = .25 * (pn[1:-1,2:] + pn[1:-1,:-2] +\
                              pn[2:,1:-1] + pn[:-2,1:-1])
        
        ##Neumann B.C. along x = L
        p[1:-1,-1] = .25 * (2*pn[1:-1,-2] + pn[2:,-1] + pn[:-2, -1])
        
        l2norm = numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))
        iterations += 1
     
    return p, iterations

In [5]:
l2_target = 1e-8
p, iterations = laplace2d(p0.copy(), l2_target)

print ("Jacobi method took {} iterations at tolerance {}".\
        format(iterations, l2_target))

Jacobi method took 19993 iterations at tolerance 1e-08


In [6]:
%%timeit
laplace2d(p0.copy(), l2_target)

1 loop, best of 3: 6.11 s per loop


In [7]:
pan = p_analytical(x,y)

In [8]:
L2_rel_error(p,pan)

6.1735513352973877e-05

In [9]:
def laplace2d_gauss_seidel(p, nx, ny, l2_target):
    
    iterations = 0
    iter_diff = l2_target+1 #init iter_diff to be larger than l2_target
    
    while iter_diff > l2_target:
        pn = p.copy()
        iter_diff = 0.0
        for j in range(1,ny-1):
            for i in range(1,nx-1):
                p[j,i] = .25 * (p[j,i-1] + p[j,i+1] + p[j-1,i] + p[j+1,i])
                iter_diff += (p[j,i] - pn[j,i])**2
        
        #Neumann 2nd-order BC
        for j in range(1,ny-1):
            p[j,-1] = .25 * (2*p[j,-2] + p[j+1,-1] + p[j-1, -1])
            
        iter_diff = numpy.sqrt(iter_diff/numpy.sum(pn**2))
        iterations += 1        
        
    return p, iterations

In [10]:
import numba
from numba import jit

In [11]:
def fib_it(n):
    a = 1
    b = 1
    for i in range(n-2):
        a, b = b, a+b
        
    return b

In [12]:
%%timeit
fib_it(500000)

1 loop, best of 3: 4.04 s per loop


In [13]:
def fib_it(n):
    a = 1
    b = 1
    for i in range(n-2):
        a, b = b, a+b
        
    return b

In [14]:
%%timeit
fib_it(500000)

1 loop, best of 3: 4.04 s per loop


In [ ]:
%%timeit
fib_it(500000)

In [ ]:
print(numba.__version__)